# Investigation of Breit–Wigner behavior

In [ ]:
%pip install -q ampform ipympl matplotlib tensorwaves zarr~=2.0

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib widget

In [ ]:
import warnings

import ampform
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import qrules
import sympy as sp
import zarr
from ampform.dynamics import phasespace
from ampform.dynamics.builder import RelativisticBreitWignerBuilder
from ampform.io import improve_latex_rendering
from ampform.sympy import cached
from IPython.display import display
from tensorwaves.function.sympy import create_parametrized_function

warnings.filterwarnings(
    "ignore",
    message=".*All-NaN slice encountered.*",
    category=RuntimeWarning,
)
improve_latex_rendering()

## Generate transitions

In [ ]:
reaction = qrules.generate_transitions(
    initial_state=("psi(2S)", [-1, +1]),
    final_state=["phi(1020)", "K+", "K-"],
    allowed_intermediate_particles=["f(0)(980)"],
    allowed_interaction_types=["strong"],
    formalism="canonical-helicity",
)

In [ ]:
(phi,) = reaction.get_intermediate_particles()
phi

## Build model

In [ ]:
model_builder = ampform.get_builder(reaction)
model_builder.config.scalar_initial_state_mass = True
model_builder.config.stable_final_state_ids = [0, 1, 2]
bw_builder = RelativisticBreitWignerBuilder(
    energy_dependent_width=True,
    form_factor=True,
    phsp_factor=phasespace.PhaseSpaceFactor,  # or: PhaseSpaceFactorSWave
)
for name in reaction.get_intermediate_particles().names:
    model_builder.dynamics.assign(name, bw_builder)
model = model_builder.formulate()

## Load phase space

In [ ]:
with zarr.storage.ZipStore("phsp.zarr.zip", mode="r") as store:
    group = zarr.open_group(store, mode="r")
    phsp = {k: v[:] for k, v in group.arrays()}
phsp

## Create numerical functions

In [ ]:
fixed_parameters = {
    s: v
    for s, v in model.parameter_defaults.items()
    if not s.name.startswith(("m_{", R"\Gamma_{"))
}
fixed_parameters

In [ ]:
full_intensity_expr = cached.unfold(model)
substituted_intensity_expr = cached.xreplace(full_intensity_expr, fixed_parameters)

In [ ]:
amplitude_expr, *_ = model.amplitudes.values()
assert isinstance(amplitude_expr, sp.Add)
amplitude_expr = amplitude_expr.args[0]
amplitude_expr.doit()

In [ ]:
substituted_amplitude_expr = amplitude_expr.xreplace(fixed_parameters).doit()
substituted_amplitude_expr = substituted_amplitude_expr.xreplace({
    s: 0 for s in amplitude_expr.free_symbols if s.name.startswith("phi")
})
substituted_amplitude_expr = substituted_amplitude_expr.xreplace({
    s: sp.pi / 2 for s in amplitude_expr.free_symbols if s.name.startswith("theta")
})
substituted_amplitude_expr

In [ ]:
remaining_parameters = {
    k: v
    for k, v in model.parameter_defaults.items()
    if k in substituted_intensity_expr.free_symbols
}
remaining_parameters

In [ ]:
amplitude_func = create_parametrized_function(
    expression=substituted_amplitude_expr,
    parameters=remaining_parameters,
    backend="numpy",
)
intensity_func = create_parametrized_function(
    expression=substituted_intensity_expr,
    parameters=remaining_parameters,
    backend="numpy",
)

## Visualize

In [ ]:
key = "m_12"
phsp[key].min(), phsp[key].max()

In [ ]:
epsilon = 1e-8
domain = {key: np.linspace(0.8, 2.9, num=200) + epsilon * 1j}
phsp[key] = phsp[key].real + epsilon * 1j

In [ ]:
sliders = {
    key: w.FloatSlider(description=f"${key}$", min=0, max=3.0, step=0.01, value=value)
    for key, value in intensity_func.parameters.items()
}
ui = w.VBox(list(sliders.values()))

In [ ]:
def update_plot(**parameters: float) -> None:
    global LINES
    amplitude_func.update_parameters(parameters)
    intensity_func.update_parameters(parameters)
    intensities = intensity_func(phsp)
    amplitudes = amplitude_func(domain)
    bin_values, bin_edges = np.histogram(
        phsp[key].real,
        bins=100,
        density=True,
        weights=intensities,
    )
    bin_values_phsp, _ = np.histogram(
        phsp[key].real,
        bins=bin_edges,
        density=True,
    )
    bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2
    bin_width = (bin_edges[1:] - bin_edges[:-1]).mean()
    x_values = domain[key].real
    y_values = amplitudes.real**2 + amplitudes.imag**2
    y_values = y_values / np.nansum(y_values) / bin_width
    if LINES is None:
        LINES = [
            ax.plot(x_values, y_values, label="Dynamics lineshape")[0],
            ax.step(bin_centers, bin_values, label="Intensity")[0],
            ax.step(
                bin_centers,
                bin_values_phsp,
                c="black",
                ls="dotted",
                label="Phase space",
            )[0],
        ]
    else:
        LINES[0].set_ydata(y_values)
        LINES[1].set_ydata(bin_values)
        LINES[2].set_ydata(bin_values_phsp)
    y_max = np.nanmax([
        np.nanmax(y_values),
        np.nanmax(bin_values),
        np.nanmax(bin_values_phsp),
    ])
    ax.set_ylim(x_values.min(), x_values.max())
    ax.set_ylim(0, 1.1 * y_max)
    fig.canvas.draw_idle()


LINES = None
fig, ax = plt.subplots()
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
ax.set_xlabel("$M(K^+K^-)$")
ax.set_ylabel("Intensity (a.u.)")
output = w.interactive_output(update_plot, controls=sliders)
fig.legend(loc="upper right")
fig.tight_layout()
display(output, ui)

In [ ]:
substituted_amplitude_expr